# Simplex Methode

## 1. Standardize the writing of our problem

In [46]:
import numpy as np
import warnings
warnings.simplefilter("ignore")


#Define the initial problem 
objective_function = np.array([5, 4, 3])  
constraints_factor = np.array([[2, 3, 1], [4, 1, 2], [3, 4, 2]])  
constrains_values = np.array([5, 11, 8]) 

def initialize_simplex(objective_function, constraints_factor, constrains_values):
    num_constraints, num_variables = constraints_factor.shape
    
    # Construction of the simplex table
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))

    # Fill in the constraint coefficient matrix
    tableau[:-1, :-1] = np.hstack((constraints_factor, np.eye(num_constraints)))

    # Fill in the constraints vector (right-hand column)
    tableau[:-1, -1] = constrains_values

    # Fill in the objective function line (last line)
    tableau[-1, :-1] = np.hstack((objective_function, np.zeros(num_constraints)))

    return tableau

simplex_matrix = initialize_simplex(objective_function, constraints_factor, constrains_values)
print(simplex_matrix)


[[ 2.  3.  1.  1.  0.  0.  5.]
 [ 4.  1.  2.  0.  1.  0. 11.]
 [ 3.  4.  2.  0.  0.  1.  8.]
 [ 5.  4.  3.  0.  0.  0.  0.]]


## 2. On doit chercher la colonne pivot 

In [47]:
def pivot_column(simplex_matrix):
    last_row = simplex_matrix[-1, :-1]
    pivot_col = np.argmax(last_row)  # Index of the most positive coefficient
    return pivot_col if last_row[pivot_col] > 0 else None


## 3. On doit chercher le pivot 

In [48]:
def find_pivot (simplex_matrix, pivot_col):
    num_rows =( simplex_matrix.shape[0] - 1)
    rhs = simplex_matrix[:-1, -1]
    column = simplex_matrix[:-1, pivot_col]
    
    ratios = np.where(column > 0, rhs / column, np.inf)
    pivot_row = np.argmin(ratios)
    
    return pivot_row if ratios[pivot_row] != np.inf else None
    

## 4. On iterre jusqu'à ce que les coeficient soit tous negatif 

In [49]:
def pivot(tableau, pivot_row, pivot_col):
    tableau[pivot_row, :] /= tableau[pivot_row, pivot_col]
    
    for i in range(tableau.shape[0]):
        if i != pivot_row:
            tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]


In [50]:
def simplex_method(objective_function, constraints_factor, constrains_values):
    tableau = initialize_simplex(objective_function, constraints_factor, constrains_values)
    
    while True:
        pivot_col = pivot_column(tableau)
        if pivot_col is None:
            break  # We've reached the optimum
        
        pivot_row = find_pivot(tableau, pivot_col)
        if pivot_row is None:
            raise ValueError("Problème non borné !")
        
        pivot(tableau, pivot_row, pivot_col)

    
    print(tableau)

    solution=[]
    for i in range(len(objective_function)) :
        if (np.sum(tableau[:-1,i])==1.0) :
            sol_index = np.where(tableau[:-1,i]==1.0)[0]
            solution.append(tableau[sol_index,-1][0])
        else : 
            solution.append(0.0)

    optimal_value = abs(tableau[-1, -1])
    return solution, optimal_value


In [51]:
objective_function = np.array([2.6, 4.0, 3.6])  
constraints_factor = np.array([[37.5, 12.5, 100.0], [50, 5, 20], [50, 20, 15]])  
constrains_values = np.array([400000.0, 80000.0 , 150000.0]) 

solution, optimal_value = simplex_method(objective_function, constraints_factor, constrains_values)
print("Solution optimale :", solution)
print("Valeur optimale de la fonction objective :", optimal_value)

[[-2.02884615e+02  0.00000000e+00  0.00000000e+00  1.00000000e+00
  -5.57692308e+00  7.69230769e-01  6.92307692e+04]
 [ 2.30769231e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   6.15384615e-02 -1.53846154e-02  2.61538462e+03]
 [ 7.69230769e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.61538462e-02  6.15384615e-02  5.53846154e+03]
 [-8.78461538e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.69230769e-02 -1.90769231e-01 -3.15692308e+04]]
Solution optimale : [0.0, 5538.461538461539, 2615.3846153846152]
Valeur optimale de la fonction objective : 31569.23076923077


### We can also simply use Scipy's simplex method, which always aims to minimize the objective function. 

### (This means inverting the coeficients of the objective function)

In [52]:
import numpy as np
from scipy.optimize import linprog

# Fonction objectif ( negative car linprog minimise )
c = [ -3 , -4]

# Contraintes
A = [[2 , 1] , [1 , 2]]
b = [100 , 80]

# Resolution
result = linprog (c , A_ub =A , b_ub =b , method ='simplex')

print (" Solution optimale :")
print (f"x1={result.x[0]:.0f},x2={result.x[1]:.0f}")
print ( f" Profit maximal = { - result .fun :.0f}eur")

 Solution optimale :
x1=40,x2=20
 Profit maximal = 200eur


In [53]:
import numpy as np
from scipy.optimize import linprog

# Fonction objectif ( negative car linprog minimise )
c = [2 , 4 , 5 , 3 , 1 , 3]

# Contraintes
A = [
[1 , 0 , 0 , 1 , 0 , 0], # Magasin 1
[0 , 1 , 0 , 0 , 1 , 0], # Magasin 2
[0 , 0 , 1 , 0 , 0 , 1], # Magasin 3
[1 , 1 , 1 , 0 , 0 , 0], # Entrepot 1
[0 , 0 , 0 , 1 , 1 , 1]
]
b = [30 , 40 , 50, 50 , 70]

# Resolution
result = linprog (c , A_ub =A , b_ub =b , method ='simplex')

print (" Solution optimale :")
print (f"x11={result.x[0]:.0f},x12={result.x[1]:.0f},x13={result.x[2]:.0f}")
print (f"x21 ={result.x[3]:.0f},x22={result.x[4]:.0f},x23={result.x[5]:.0f}")
print (f"Cout minimal={result.fun:.0f}eur")

 Solution optimale :
x11=0,x12=0,x13=0
x21 =0,x22=0,x23=0
Cout minimal=0eur


In [54]:
# Fonction objectif ( negative car linprog minimise )
c = [2 , 4 , 5 , 3 ,1 , 3]



A_eq = [
[1 , 0 , 0 , 1 , 0 , 0] , # Magasin 1
[0 , 1 , 0 , 0 , 1 , 0] , # Magasin 2
[0 , 0 , 1 , 0 , 0 , 1] # Magasin 3
]
b_eq = [30 , 40 , 50]

# Contraintes
A_ub = [
[1 , 1 , 1 , 0 , 0 , 0] , # Entrepot 1
[0 , 0 , 0 , 1 , 1 , 1] # Entrepot 2
 ]
b_ub = [50 , 70]

# Resolution
result = linprog (c , A_ub = A_ub , b_ub = b_ub , A_eq = A_eq , b_eq = b_eq , method ='simplex')

print (" Solution optimale :")
print (f"x11={result.x[0]:.0f},x12={result.x[1]:.0f},x13={result.x[2]:.0f}")
print (f"x21 ={result.x[3]:.0f},x22={result.x[4]:.0f},x23={result.x[5]:.0f}")
print (f"Cout minimal={result.fun:.0f}eur")

 Solution optimale :
x11=30,x12=0,x13=20
x21 =0,x22=40,x23=30
Cout minimal=290eur


In [55]:
c = [ -2 , -2]
A = [[1 , 1]]
b = [1]
result = linprog (c , A_ub =A , b_ub =b , method ='simplex')
print(f"Solution:x1={result.x[0]:.0f},x2={result.x[1]:.0f}")
print(f"Valeur optimale={-result.fun:.0f}")

Solution:x1=1,x2=0
Valeur optimale=2


In [56]:
objective_function = np.array([2.6, 4.0, 3.6])  
constraints_factor = np.array([[37.5, 12.5, 100.0], [50, 5, 20], [50, 20, 15]])  
constrains_values = np.array([400000.0, 80000.0 , 150000.0]) 

solution, optimal_value = simplex_method(objective_function, constraints_factor, constrains_values)
print("Solution optimale :", solution)
print("Valeur optimale de la fonction objective :", optimal_value)

[[-2.02884615e+02  0.00000000e+00  0.00000000e+00  1.00000000e+00
  -5.57692308e+00  7.69230769e-01  6.92307692e+04]
 [ 2.30769231e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   6.15384615e-02 -1.53846154e-02  2.61538462e+03]
 [ 7.69230769e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.61538462e-02  6.15384615e-02  5.53846154e+03]
 [-8.78461538e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.69230769e-02 -1.90769231e-01 -3.15692308e+04]]
Solution optimale : [0.0, 5538.461538461539, 2615.3846153846152]
Valeur optimale de la fonction objective : 31569.23076923077
